In [1]:
#!pip install transformers
#!pip install tqdm
#!pip install time

In [2]:
#GPU
!nvidia-smi

# Check that PyTorch sees it
import torch
torch.cuda.empty_cache()
torch.cuda.is_available()

Fri Mar 18 12:34:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 472.12       Driver Version: 472.12       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:0A:00.0  On |                  N/A |
|  0%   45C    P8    26W / 400W |   3919MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

True

In [4]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base", max_len=512)

In [75]:
from transformers import RobertaForMaskedLM

#model = RobertaForMaskedLM(config=config)

model = RobertaForMaskedLM.from_pretrained("roberta-base")

#model.num_parameters()

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at C:\Users\Luca/.cache\huggingface\transformers\733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_model.b

In [76]:
%%time
from transformers import LineByLineTextDataset
#from tqdm import tqdm

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="merged_files/merged_files/merged_file_web.txt",
    block_size=128,
)

Creating features from dataset file at merged_files/merged_files/merged_file_web.txt


Wall time: 14min 48s


In [77]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [78]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="models/web_model",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_gpu_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    #load_best_model_at_end = True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    #callbacks = [EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.01)]
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [79]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 89063
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 27835


Step,Training Loss
500,1.666300
1000,1.612100
1500,1.608200
2000,1.610800
2500,1.599500
3000,1.599700
3500,1.616100
4000,1.600900
4500,1.587200
5000,1.584800


Saving model checkpoint to models/web_model\checkpoint-10000
Configuration saved in models/web_model\checkpoint-10000\config.json
Model weights saved in models/web_model\checkpoint-10000\pytorch_model.bin
Saving model checkpoint to models/web_model\checkpoint-20000
Configuration saved in models/web_model\checkpoint-20000\config.json
Model weights saved in models/web_model\checkpoint-20000\pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




Wall time: 48min 18s


TrainOutput(global_step=27835, training_loss=1.4837594516234058, metrics={'train_runtime': 2898.5097, 'train_samples_per_second': 153.636, 'train_steps_per_second': 9.603, 'total_flos': 2.930901494691456e+16, 'train_loss': 1.4837594516234058, 'epoch': 5.0})

In [80]:
trainer.save_model("models/web_model")

Saving model checkpoint to models/web_model
Configuration saved in models/web_model\config.json
Model weights saved in models/web_model\pytorch_model.bin


In [82]:
#test out the model
from transformers import pipeline

fill_mask_acad = pipeline(
    "fill-mask",
    model = "models/acad_model",
    tokenizer = tokenizer
)

fill_mask_blog = pipeline(
    "fill-mask",
    model = "models/blog_model",
    tokenizer = tokenizer
)

fill_mask_fic = pipeline(
    "fill-mask",
    model = "models/fic_model",
    tokenizer = tokenizer
)

fill_mask_mag = pipeline(
    "fill-mask",
    model = "models/mag_model",
    tokenizer = tokenizer
)

fill_mask_news = pipeline(
    "fill-mask",
    model = "models/news_model",
    tokenizer = tokenizer
)

fill_mask_spok = pipeline(
    "fill-mask",
    model = "models/spok_model",
    tokenizer = tokenizer
)

fill_mask_tvm = pipeline(
    "fill-mask",
    model = "models/tvm_model",
    tokenizer = tokenizer
)

fill_mask_web = pipeline(
    "fill-mask",
    model = "models/web_model",
    tokenizer = tokenizer
)

fill_mask_base = pipeline(
    "fill-mask",
    model = RobertaForMaskedLM.from_pretrained("roberta-base"),
    tokenizer = tokenizer
)

loading configuration file models/acad_model\config.json
Model config RobertaConfig {
  "_name_or_path": "models/acad_model",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading configuration file models/acad_model\config.json
Model config RobertaConfig {
  "_name_or_path": "models/acad_model",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
 

loading configuration file models/news_model\config.json
Model config RobertaConfig {
  "_name_or_path": "models/news_model",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file models/news_model\pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForMaskedLM.

All the weights of RobertaForMaskedLM were initialized from the model checkpoint at models/news_model.


In [49]:
fill_mask_base("The <mask> are alive.")

[{'score': 0.04573606699705124,
  'token': 1462,
  'token_str': ' dead',
  'sequence': 'The dead are alive.'},
 {'score': 0.042372267693281174,
  'token': 408,
  'token_str': ' children',
  'sequence': 'The children are alive.'},
 {'score': 0.021926352754235268,
  'token': 7149,
  'token_str': ' survivors',
  'sequence': 'The survivors are alive.'},
 {'score': 0.021217139437794685,
  'token': 2786,
  'token_str': ' boys',
  'sequence': 'The boys are alive.'},
 {'score': 0.02014077454805374,
  'token': 1159,
  'token_str': ' kids',
  'sequence': 'The kids are alive.'}]

In [50]:
fill_mask_acad("The <mask> are alive.")

[{'score': 0.09346634894609451,
  'token': 1462,
  'token_str': ' dead',
  'sequence': 'The dead are alive.'},
 {'score': 0.039698731154203415,
  'token': 7723,
  'token_str': ' birds',
  'sequence': 'The birds are alive.'},
 {'score': 0.036720436066389084,
  'token': 29171,
  'token_str': ' dinosaurs',
  'sequence': 'The dinosaurs are alive.'},
 {'score': 0.028980573639273643,
  'token': 31932,
  'token_str': ' ghosts',
  'sequence': 'The ghosts are alive.'},
 {'score': 0.02068193443119526,
  'token': 408,
  'token_str': ' children',
  'sequence': 'The children are alive.'}]

In [51]:
fill_mask_blog("The <mask> are alive.")

[{'score': 0.08084041625261307,
  'token': 1462,
  'token_str': ' dead',
  'sequence': 'The dead are alive.'},
 {'score': 0.036048341542482376,
  'token': 44486,
  'token_str': ' undead',
  'sequence': 'The undead are alive.'},
 {'score': 0.029841721057891846,
  'token': 37132,
  'token_str': ' zombies',
  'sequence': 'The zombies are alive.'},
 {'score': 0.026773858815431595,
  'token': 82,
  'token_str': ' people',
  'sequence': 'The people are alive.'},
 {'score': 0.018611673265695572,
  'token': 3122,
  'token_str': ' animals',
  'sequence': 'The animals are alive.'}]

In [53]:
fill_mask_fic("The <mask> are alive.")

[{'score': 0.14949068427085876,
  'token': 408,
  'token_str': ' children',
  'sequence': 'The children are alive.'},
 {'score': 0.053679581731557846,
  'token': 1462,
  'token_str': ' dead',
  'sequence': 'The dead are alive.'},
 {'score': 0.04021308198571205,
  'token': 3122,
  'token_str': ' animals',
  'sequence': 'The animals are alive.'},
 {'score': 0.020997924730181694,
  'token': 2786,
  'token_str': ' boys',
  'sequence': 'The boys are alive.'},
 {'score': 0.01977924071252346,
  'token': 1972,
  'token_str': ' girls',
  'sequence': 'The girls are alive.'}]

In [54]:
fill_mask_mag("The <mask> are alive.")

[{'score': 0.038753196597099304,
  'token': 1462,
  'token_str': ' dead',
  'sequence': 'The dead are alive.'},
 {'score': 0.029477659612894058,
  'token': 44486,
  'token_str': ' undead',
  'sequence': 'The undead are alive.'},
 {'score': 0.027496540918946266,
  'token': 29171,
  'token_str': ' dinosaurs',
  'sequence': 'The dinosaurs are alive.'},
 {'score': 0.021867072209715843,
  'token': 11656,
  'token_str': ' spirits',
  'sequence': 'The spirits are alive.'},
 {'score': 0.01568089984357357,
  'token': 31932,
  'token_str': ' ghosts',
  'sequence': 'The ghosts are alive.'}]

In [62]:
fill_mask_news("The <mask> are alive.")

[{'score': 0.02851683273911476,
  'token': 21278,
  'token_str': ' sharks',
  'sequence': 'The sharks are alive.'},
 {'score': 0.026806717738509178,
  'token': 18018,
  'token_str': ' whales',
  'sequence': 'The whales are alive.'},
 {'score': 0.02193547785282135,
  'token': 21684,
  'token_str': ' Beatles',
  'sequence': 'The Beatles are alive.'},
 {'score': 0.021713774651288986,
  'token': 8241,
  'token_str': ' Russians',
  'sequence': 'The Russians are alive.'},
 {'score': 0.02157757431268692,
  'token': 24162,
  'token_str': ' rats',
  'sequence': 'The rats are alive.'}]

In [83]:
fill_mask_spok("The <mask> are alive.")

[{'score': 0.0728221908211708,
  'token': 1462,
  'token_str': ' dead',
  'sequence': 'The dead are alive.'},
 {'score': 0.059811994433403015,
  'token': 408,
  'token_str': ' children',
  'sequence': 'The children are alive.'},
 {'score': 0.0332520566880703,
  'token': 26445,
  'token_str': ' hostages',
  'sequence': 'The hostages are alive.'},
 {'score': 0.030539248138666153,
  'token': 21684,
  'token_str': ' Beatles',
  'sequence': 'The Beatles are alive.'},
 {'score': 0.027235785499215126,
  'token': 2786,
  'token_str': ' boys',
  'sequence': 'The boys are alive.'}]

In [84]:
fill_mask_tvm("The <mask> are alive.")

[{'score': 0.15919719636440277,
  'token': 408,
  'token_str': ' children',
  'sequence': 'The children are alive.'},
 {'score': 0.07739009708166122,
  'token': 26445,
  'token_str': ' hostages',
  'sequence': 'The hostages are alive.'},
 {'score': 0.05158675089478493,
  'token': 1462,
  'token_str': ' dead',
  'sequence': 'The dead are alive.'},
 {'score': 0.040882959961891174,
  'token': 604,
  'token_str': ' men',
  'sequence': 'The men are alive.'},
 {'score': 0.03986725956201553,
  'token': 2786,
  'token_str': ' boys',
  'sequence': 'The boys are alive.'}]

In [85]:
fill_mask_web("The <mask> are alive.")

[{'score': 0.024046516045928,
  'token': 29171,
  'token_str': ' dinosaurs',
  'sequence': 'The dinosaurs are alive.'},
 {'score': 0.02172001264989376,
  'token': 12129,
  'token_str': ' robots',
  'sequence': 'The robots are alive.'},
 {'score': 0.02050301618874073,
  'token': 27056,
  'token_str': ' monsters',
  'sequence': 'The monsters are alive.'},
 {'score': 0.020174715667963028,
  'token': 82,
  'token_str': ' people',
  'sequence': 'The people are alive.'},
 {'score': 0.019408848136663437,
  'token': 37132,
  'token_str': ' zombies',
  'sequence': 'The zombies are alive.'}]